In [4]:
import pandas as pd

In [5]:
data = pd.read_csv('train.csv')

In [6]:
data.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [10]:
data = data.dropna()

In [29]:
X = data.drop('label',axis=1)
y = data['label']

In [30]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [31]:
#define the vocabulary size
voc_size = 5000

In [32]:
#resetting the index as we drop some of the row which has NaN value
messages = X.copy()
messages.head()

,id,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...


In [33]:
messages.reset_index(inplace=True)

In [34]:
import re 
import nltk

In [35]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\91859\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [36]:
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

In [37]:
ps = PorterStemmer()

In [38]:
corpus = []

In [39]:
#removing all necessary words from message
for i in range(len(messages)):
    review = re.sub('[^a-zA-Z]',' ',messages['title'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [42]:
onehot_repr = [one_hot(words,voc_size) for words in corpus]
onehot_repr

[[909, 1977, 660, 3758, 3797, 4121, 4871, 3785, 1486, 3677],
 [909, 3617, 4234, 4459, 77, 2414, 2322],
 [901, 3529, 3902, 3693],
 [2937, 4215, 3753, 3971, 1203, 4939],
 [4686, 77, 3336, 2202, 65, 1659, 77, 2633, 2538, 4446],
 [4606,
  596,
  4278,
  617,
  2604,
  3959,
  1873,
  3300,
  4103,
  3816,
  3451,
  1154,
  3852,
  3370,
  2322],
 [4472, 4899, 579, 831, 2108, 2834, 4920, 2916, 2033, 4032, 3188],
 [2507, 3720, 563, 322, 3648, 2869, 3959, 4087, 2033, 4032, 3188],
 [4959, 251, 4200, 2878, 1691, 877, 1650, 4843, 3959, 4861],
 [1533, 4452, 2926, 956, 3670, 2390, 4940, 2897],
 [1634, 1872, 3183, 16, 3697, 3507, 1707, 528, 1419, 4123, 4684],
 [3971, 261, 3797, 877, 3959, 3648],
 [2584, 4767, 3239, 3350, 940, 861, 242, 2506, 2933],
 [1110, 753, 483, 1398, 2838, 2672, 1393, 2033, 4032, 3188],
 [285, 1032, 673, 1336, 3660, 2033, 4032, 3188],
 [4290, 1085, 4893, 3314, 329, 3211, 3509, 2352, 3420, 194],
 [4208, 1594, 3617],
 [382, 2545, 1791, 160, 3959, 4555, 729, 2322],
 [2, 1714, 423

In [44]:
#set the sentence Length
set_length = 20
embedded_docs = pad_sequences(onehot_repr,padding='pre', maxlen=set_length)
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,  909,
       1977,  660, 3758, 3797, 4121, 4871, 3785, 1486, 3677])

In [49]:
#creating Model
embedded_vector_features = 40
model = Sequential()
model.add(Embedding(voc_size,embedded_vector_features,input_length=set_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam', metrics=['accuracy'])

print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 20, 40)            200000    
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               56400     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [50]:
import numpy as np
x_final = np.array(embedded_docs)
y_final = np.array(y)

In [51]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(x_final,y_final,test_size=0.3,random_state=42)

In [52]:
#train the model
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
200/200 [==============================] - 9s 45ms/step - loss: 0.3223 - accuracy: 0.8557 - val_loss: 0.1971 - val_accuracy: 0.9163
Epoch 2/10
200/200 [==============================] - 7s 36ms/step - loss: 0.1348 - accuracy: 0.9492 - val_loss: 0.2067 - val_accuracy: 0.9203
Epoch 3/10
200/200 [==============================] - 7s 37ms/step - loss: 0.0964 - accuracy: 0.9642 - val_loss: 0.2472 - val_accuracy: 0.9178
Epoch 4/10
200/200 [==============================] - 8s 38ms/step - loss: 0.0708 - accuracy: 0.9752 - val_loss: 0.2972 - val_accuracy: 0.9136
Epoch 5/10
200/200 [==============================] - 8s 39ms/step - loss: 0.0554 - accuracy: 0.9813 - val_loss: 0.3215 - val_accuracy: 0.9107
Epoch 6/10
200/200 [==============================] - 8s 40ms/step - loss: 0.0409 - accuracy: 0.9868 - val_loss: 0.3600 - val_accuracy: 0.9021
Epoch 7/10
200/200 [==============================] - 8s 41ms/step - loss: 0.0275 - accuracy: 0.9916 - val_loss: 0.3843 - val_accuracy: 0.9070

### Performance matrix and Accuracy

In [74]:
y_pred = model.predict_classes(X_test)
y_pred

array([1, 0, 0, ..., 0, 1, 0], dtype=int64)

In [54]:
from sklearn.metrics import confusion_matrix,accuracy_score

In [55]:
confusion_matrix(y_test,y_pred)

array([[2817,  290],
       [ 220, 2159]], dtype=int64)

In [56]:
accuracy_score(y_test,y_pred)

0.907036091870215

In [71]:
#Adding Dropout When Creating model
from tensorflow.keras.layers import Dropout
model= Sequential()
model.add(Embedding(voc_size,embedded_vector_features,input_length=set_length))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(2,activation='softmax'))
model.compile(loss='binary_crossentropy',optimizer="adam",metrics=['accuracy'])

print(model.summary())

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 20, 40)            200000    
_________________________________________________________________
dropout_2 (Dropout)          (None, 20, 40)            0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 100)               56400     
_________________________________________________________________
dropout_3 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 2)                 202       
Total params: 256,602
Trainable params: 256,602
Non-trainable params: 0
_________________________________________________________________
None


In [59]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=

SyntaxError: unexpected EOF while parsing (<ipython-input-59-49d8b8856e0f>, line 1)

In [72]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
200/200 [==============================] - 9s 44ms/step - loss: 0.6932 - accuracy: 0.4929 - val_loss: 0.6931 - val_accuracy: 0.4623
Epoch 2/10
200/200 [==============================] - 7s 36ms/step - loss: 0.6931 - accuracy: 0.5027 - val_loss: 0.6931 - val_accuracy: 0.5242
Epoch 3/10
200/200 [==============================] - 8s 38ms/step - loss: 0.6931 - accuracy: 0.4985 - val_loss: 0.6931 - val_accuracy: 0.5735
Epoch 4/10
200/200 [==============================] - 7s 37ms/step - loss: 0.6931 - accuracy: 0.5029 - val_loss: 0.6931 - val_accuracy: 0.5844
Epoch 5/10
200/200 [==============================] - 7s 37ms/step - loss: 0.6931 - accuracy: 0.4954 - val_loss: 0.6931 - val_accuracy: 0.5419
Epoch 6/10
200/200 [==============================] - 8s 38ms/step - loss: 0.6931 - accuracy: 0.4954 - val_loss: 0.6931 - val_accuracy: 0.4121
Epoch 7/10
200/200 [==============================] - 7s 36ms/step - loss: 0.6931 - accuracy: 0.4964 - val_loss: 0.6931 - val_accuracy: 0.5089

In [78]:
y_pred = model.predict(X_test)
y_pred= np.argmax(y_pred, -1)


In [79]:
confusion_matrix(y_test,y_pred)

array([[2745,  362],
       [1819,  560]], dtype=int64)